In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.image as image
import cv2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential
import numpy as np
from sklearn import preprocessing
from random import shuffle
from numpy import argmax

# **Reading dataset**

In [ ]:

train = glob.glob("../input/chess-positions/train/*.jpeg")
test = glob.glob("../input/chess-positions/test/*.jpeg")


In [ ]:
train_size = 3000
test_size = 1000
shuffle(train)
shuffle(test)
train = train[:train_size]
test = test[:test_size]


In [ ]:
def fen_label_extraction(data):
    """
This method take dataset as parameter and create labels for all items
    :param data: list of images in FEN format
    :return: labels contain labels for each 64 images in 8X8 chess board
    """

    labels = []
    for img in data:
        base = os.path.basename(img)
        labels.append(os.path.splitext(base)[0])
    return labels
    
def getX(data):
    """
This method take dataset as parameter and return list of images
    :param data: list of images in FEN format
    :return: list of images in Pixel format
    """

    X = [cv2.imread(i) for i in data]
    return X

def split_chess_board(board):
    """
This method takes a chess board image as input and generate 64 images (8X8 chess board = 64 sub images)
    :param board: Image of chess board
    :return: 64 images reshaped into 25X25X3 shape
    """

    temp = []
    for x in range(0,8):
        temp2 = board[(x*25):((x+1)*25),:]
        for y in range(0,8):
            temp.append(temp2[:,(y*25):((y+1)*25)].flatten().reshape(25,25,3))
    return temp

def split_labels(y_train):
    """
This split the image name in FEN format into labels for 64 images
    :param y:  The label of image in FEN
    :return: seperate labels of size 64
    """

    labels = []
    for i in y_train:
        for ches_row in i.split('-'):
            for item in ches_row:
                if(item.isdigit()):
                    empty_pos = int(item)
                    for i in range(empty_pos):
                        labels.append('1')
                else:
                    labels.append(item)
    return labels

def prepare_test_image(img):
    """
Given a random image of chess board, it will generate images for making prediction

    :param img: image of chess board
    :return: preprocessed image for making prediction and copy of original image
    """

    test_img = cv2.imread(img)
    board  =cv2.resize(test_img, (200, 200))
    temp = []
    for x in range(0,8):
        temp2 = board[(x*25):((x+1)*25),:]
        for y in range(0,8):
            temp.append(temp2[:,(y*25):((y+1)*25)].flatten().reshape(25,25,3))
            
    tes = np.array(temp).astype("float32") / 255
    return tes,test_img

def preprocess(X):
    """
Given an image of chess board, it will preprocess and generate 64 images

    :param X: image of chess board
    :return: list of 64 images
    """

    temp=[]
    for i in X:
        img  =cv2.resize(i, (200, 200))
        temp.extend(split_chess_board(img))  
    return temp

def decode_fen(prediction):
    """
It will recreate the FEN label from predicted label
    :param prediction: predicted label in encoded format
    :return: FEN string
    """

    predicted_fen = []
    for i in prediction[0]:
        row = le.inverse_transform(i)
        fen = ''
        count = 0
        digit_flag = False
        for j in range(len(row)):
            if row[j].isdigit():
                count = count+1
                digit_flag = True
                if(j==7):
                    fen = fen+ str(count)
            else:
                if digit_flag:
                    fen = fen+ str(count)
                    fen = fen+row[j]
                else:
                    fen = fen+row[j]
                digit_flag = False
                count = 0
        predicted_fen.append(fen)
    return '-'.join(predicted_fen)s


In [ ]:
y_train_global = np.array(fen_label_extraction(train))
y_test_global = np.array(fen_label_extraction(test))
X_train_global = np.array(getX(train))
X_test_global = np.array(getX(test))

In [ ]:
y_train = y_train_global
y_test = y_test_global
X_train = X_train_global
X_test = X_test_global

# **Visualize Data**

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(15,15))
for i in range(4):
    ax[i].set_title(y_train[i])
    ax[i].imshow(X_train[i])

# **Preprocessing Train data**

In [ ]:
X_train = preprocess(X_train)
fig, ax = plt.subplots(1,8, figsize=(15,15))
for i in range(8):
    ax[i].imshow(X_train[i])


# **Preprocessing train labels**

In [ ]:
y = np.array(split_labels(y_train), dtype='str')
print(y_train[0])
print(y[0],y[1],y[2],y[3],y[4],y[5],y[6],y[7])

In [ ]:
print("Length of training data: ",len(X_train))
print("Length of training labels: ",len(y))

In [ ]:
unique_labels = list(set(y))
print('unique_labels: ',unique_labels)

# **Label encoder for train**

In [ ]:
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y)
print('Encoded classes: ',list(le.classes_))
print(y_train[:7])
print(y[:7])


# **Prepocessing Test Data**

In [ ]:
X_test = preprocess(X_test)

In [ ]:
from keras.utils import np_utils
y_t = np.array(split_labels(y_test),dtype='str')
y_test = le.transform(y_t)


In [ ]:
plt.imshow(X_test[3])
print('y_test:',y_test[3])

In [ ]:
X_train = (np.array(X_train))
y_train = (np.array(y_train))
X_train = X_train.astype("float32") / 255

print("x_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)


X_test = (np.array(X_test))
y_test = (np.array(y_test))
X_test = X_test.astype("float32") / 255

print("x_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)




# **Designing Neural Net**

In [ ]:
model = Sequential()
model.add(Conv2D(25, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(25, 25, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(50, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(13, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
model.summary()


In [ ]:
history = model.fit(X_train,y_train, epochs = 2, validation_data=(X_test, y_test))


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [22]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# **Make Prediction**

In [23]:
from random import randint

random_index = randint(0,999)
test_image,test_img = prepare_test_image(test[random_index])


In [24]:
res = (
  model.predict(test_image)
  .argmax(axis=1)
  .reshape(-1, 8, 8)
)
res

In [25]:
plt.imshow(test_img)


In [26]:
print("Actual label:", test[random_index].split('/')[-1] )
print("predicted FEN:",decode_fen(res))
